In [2]:
#初期設定
import os
import pandas as pd
import numpy as np
import time
import lxml
import re
import requests
from bs4 import BeautifulSoup as bs4


In [3]:
# https://web-kiwami.com/python-beautyfulsoup4.html
# http://kondou.com/BS4/
# bs4参考

In [4]:
# all_year
this_year = 2022 #年変わったら変更
get_year = list(range(2011,this_year+1))
df_result = pd.DataFrame()
dict1 = {}
for year in get_year:
    if 2011 <= year <= 2017:
        print(year)
        # 単年のデータ取得2011~2017
        url =  rf"https://www.ipokiso.com/company/{year}.html"
        html_res = requests.get(url)
        soup = bs4(html_res.content, 'html.parser')
        find_all_list = soup.find_all(href=re.compile("company/[0-9]{4}/"))
        url_dict = {
            find_all_list[i].text:"https://www.ipokiso.com/" + find_all_list[i]["href"]
            for i in range(len(find_all_list))
        }
        dict1.update(url_dict)
        df_url = pd.read_html(url)
        time.sleep(np.random.randint(100,120)/100)
        for i in range(len(df_url)):
            df_url[i]["上場年"] = f"{year}"
            df_result = pd.concat([df_result, df_url[i]])
            df_result = df_result.reset_index(drop=True)
        # df_result.drop(df_result[df_result['企業名'] == "企業名"].index , inplace=True)
        # df_result.drop(df_result[df_result['初値'] == "初値"].index , inplace=True)
    else:
        # 単年のデータ取得2018~2022
        print(year)
        url = rf"https://www.ipokiso.com/company/{year}.html"
        if year == 2022:
            url = r"https://www.ipokiso.com/company/index.html"
        html_res = requests.get(url)
        soup = bs4(html_res.content, 'html.parser')
        find_all_list = soup.find_all(href=re.compile("company/[0-9]{4}/"))
        url_dict = {
            find_all_list[i].text:"https://www.ipokiso.com/" + find_all_list[i]["href"]
            for i in range(len(find_all_list))
        }
        dict1.update(url_dict)
        df_url = pd.read_html(url)
        time.sleep(np.random.randint(100,120)/100)
        for i in range(0, len(df_url) , 2):
            df_url_con = pd.concat([df_url[i], df_url[i+1]], axis=1)
            df_url_con["上場年"] = f"{year}"
            if year == 2019:
                df_url_con = df_url_con.rename(columns={'上場 市場': '上場市場'})
            if year >= 2020:
                df_url_con = df_url_con.rename(columns={'申し込み期間': '申し込み 期間', '初値上昇率': '初値 上昇率'})
            df_result = pd.concat([df_result, df_url_con],ignore_index=True)
            df_result = df_result.reset_index(drop=True)
        df_result.drop(df_result[df_result['企業名'] == "企業名"].index , inplace=True)
        df_result.drop(df_result[df_result['初値'] == "初値"].index , inplace=True)
    if year != this_year:
        time.sleep(np.random.randint(100,120)/100)

2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022


In [14]:
# dict作成
# 空白削除
del dict1[" "]
del dict1[""]
del dict1["\n"]
# dictをdf化
df_dict = pd.DataFrame.from_dict(dict1, orient='index')
# カラム名変更
df_dict = df_dict.rename(columns={0:"url"})
# 間違えているurlを変更
df_dict.url[df_dict.url == "https://www.ipokiso.com/https://www.ipokiso.com/company/2013/zigexn.html"] = "https://www.ipokiso.com/company/2013/zigexn.html"

In [16]:
# code_listの作成（詳細情報の取得ループ）全てのcode取得ok！
def make_company_info(url):
    html_res = requests.get(url)
    time.sleep(np.random.randint(100,120)/100)
    soup = bs4(html_res.content, 'html.parser')
    try:
        code = re.search(r"[0-9]{4}", soup.title.text).group()
    except AttributeError:
        f = soup.find_all('h1',text=re.compile(r"[0-9]{4}"))[0].text
        code = re.search(r"[0-9]{4}", f).group()
    code_list.append(code)
    print(url)
    print(code)
code_list = []
url_list = df_dict.url.values
[make_company_info(url) for url in url_list]
print(code_list)


https://www.ipokiso.com//company/2011/aisei.html
3170
https://www.ipokiso.com//company/2011/unico.html
3169
https://www.ipokiso.com//company/2011/starflyer.html
9206
https://www.ipokiso.com//company/2011/nitta-gelatin.html
4977
https://www.ipokiso.com//company/2011/chiome.html
4583
https://www.ipokiso.com//company/2011/j-material.html
6055
https://www.ipokiso.com//company/2011/w-scope.html
6619
https://www.ipokiso.com//company/2011/nexon.html
3659


KeyboardInterrupt: 

- IPO当選株数のデータ取得

In [92]:
df_html_table = pd.read_html("https://www.ipokiso.com/company/2022/tms-japan.html")[4]

cols_0 = df_html_table.T.iloc[0,:].values
val = df_html_table.T.iloc[1:,:].values.flatten()
df_ipo_stock_info = pd.DataFrame([val],columns=cols)


In [110]:
stock_info = [
    int(re.sub( r'\D', '', (arr.split("株")[i]))) 
    for i in range(3) 
    for arr in val
]
cols_1 = np.ravel([[c,c+"_国内",c+"_海外"] for c in cols])
df_stock_info = pd.DataFrame([stock_info],columns=cols_1)

In [114]:
df_result[df_stock_info.columns] = df_stock_info


In [115]:
df_result

,上場日,企業名,銘柄 コード,上場市場,申し込み 期間,当たり本数,想定価格,仮条件,公募価格,初値,...,総合評価,公募株数,公募株数_国内,公募株数_海外,売出株数（OA含む）,売出株数（OA含む）_国内,売出株数（OA含む）_海外,当選株数合計,当選株数合計_国内,当選株数合計_海外
0,12/22,アイセイ薬局,3170,JASDAQ スタンダード,12/6 ～12/12,"2,490本","3,460円","3,300円 ～3,460円","3,460円","3,460円",...,NaN,3432800.0,858100.0,4290900.0,1302200.0,646400.0,1948600.0,2130600.0,21170.0,2342300.0
1,12/22,ミサワ,3169,東証 マザーズ,12/7 ～12/13,"6,000本",860円,760円 ～860円,860円,"1,209円",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,12/21,スターフライヤー,9206,東証 2部,12/5 ～12/9,"2,000本","3,180円","2,900円 ～3,180円","3,180円","5,500円",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,12/20,新田ゼラチン,4977,東証,12/5 ～12/9,"36,280本",570円,500円 ～570円,500円,462円,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,12/20,カイオム・ バイオサイエンス,4583,東証 マザーズ,12/2 ～12/8,"7,770本","2,700円","2,100円 ～2,700円","2,700円","2,480円",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1052,2/17,エッジテクノロジー (4268),NaN,東証マザーズ,2/1～2/7,"34,759本（多い）",690円（低価格）,300円～350円,350円,694円,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1054,2/9,ライトワークス (4267),NaN,東証マザーズ,1/21～1/27,"3,680本","2,100円","1,900円～2,100円","2,100円","3,000円",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1055,2/4,ビッグツリーテクノロジー＆コンサルティング (4266) 上場中止,NaN,東証マザーズ,1/20～1/26,"42,734本（多い）","5,000円",-,-,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1056,2/4,セイファート (9213),NaN,JASDAQスタンダード,1/19～1/25,"8,574本","1,120円","1,060円～1,120円","1,120円","1,030円",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [117]:
"(" in "エッジテクノロジー (4268)"

True

以後チェック用code

In [ ]:
# URLdict（単年）作成 確認用
year = "2020"
url = f'https://www.ipokiso.com/company/{year}.html'
html_res = requests.get(url)
time.sleep(np.random.randint(100,120)/100)
soup = bs4(html_res.content, 'html.parser')
find_all_list = soup.find_all(href=re.compile(f"company/{year}/"))
url_dict = {
    find_all_list[i].text:"https://www.ipokiso.com/" + soup.find_all(href=re.compile(f"company/{year}/"))[i]["href"]
    for i in range(len(find_all_list))
}
url_dict

In [ ]:
# 単年のデータ取得2011~2017　確認用
df_result = pd.DataFrame()
year = "2011"
url =  rf"https://www.ipokiso.com/company/{year}.html"
html_res = requests.get(url)
time.sleep(np.random.randint(100,120)/100)
soup = bs4(html_res.content, 'html.parser')
find_all_list = soup.find_all(href=re.compile(f"company/{year}/"))
print(len(find_all_list))
df_url = pd.read_html(url)
time.sleep(np.random.randint(100,120)/100)
for i in range(len(df_url)):
    df_url[i]["上場年"] = f"{year}"
    df_result = pd.concat([df_result, df_url[i]])
    df_result = df_result.reset_index(drop=True)
df_result.drop(df_result[df_result['企業名'] == "企業名"].index , inplace=True)
df_result.drop(df_result[df_result['初値'] == "初値"].index , inplace=True)
df_result

In [ ]:
# 単年のデータ取得2018~2022　確認用
df_result = pd.DataFrame()
year = "2022"
url =  rf"https://www.ipokiso.com/company/{year}.html"
if year == "2022":
    url =  r"https://www.ipokiso.com/company/index.html"
html_res = requests.get(url)
time.sleep(np.random.randint(100,120)/100)
soup = bs4(html_res.content, 'html.parser')
find_all_list = soup.find_all(href=re.compile(f"company/{year}/"))
print(len(find_all_list))
df_url = pd.read_html(url)
time.sleep(np.random.randint(100,120)/100)
for i in range(0, len(df_url) , 2):
    df_url_con = pd.concat([df_url[i], df_url[i+1]], axis=1)
    df_url_con["上場年"] = f"{year}"
    df_result = pd.concat([df_result, df_url_con],ignore_index=True)
    df_result = df_result.reset_index(drop=True)
df_result.drop(df_result[df_result['企業名'] == "企業名"].index , inplace=True)
df_result.drop(df_result[df_result['初値'] == "初値"].index , inplace=True)
df_result

In [ ]:
"""ArithmeticErrordf = df.drop_duplicates()
#カラムの重複した行を消す。
df = df[(df['申込日'] != "申込日").values].reset_index(drop=1)
#index番号をリセットする。
col = ['申込日', '株数', '前週比', '金額', '前週比.1', '株数.1', '前週比.1.1', '金額.1', '前週比.1.2','評価損益率(%)', '信用倍率']
df.columns = col
#columns（カラムズ）パラメータで列名を自分で指定し修正する。
#2重になってたカラムも一つに統一できる。MultiIndex⇒Indexにパラメータを変更。
"""

In [ ]:
# df.to_csv(r"C:\Users\xxp2p\OneDrive\ドキュメント\MEGA_saya\Traders_web\margin_transition\csv\margin_transition.csv", encoding='utf-8_sig')
#文字化け改善 encoding='utf-8_sig

In [ ]:
#読み込み
# pd.read_csv(r"C:\Users\xxp2p\OneDrive\ドキュメント\MEGA_saya\Traders_web\margin_transition\csv\margin_transition.csv", header=None, na_values=['-'])

In [ ]:
#Pythonの文字列を拡張するf,r,b,uについて
#f… .format()を使うことなく文字列の中に変数を埋め込むことができる。
#r… raw string　通常はバックスラッシュ（改行）があった場合エスケープシーケンスが働くが、raw stringを用いることで無視。
#b…バイト列リテラル str型ではなくバイト型のインスタンスを作成します。
#u…Unicodeに変換する
#fとrだけ使えればOK！

In [ ]:
#縦方向に結合する方法|append(), concat()  参考URL https://obgynai.com/python-pandas-index-merge-join/
#append：新しい行を追加するメソッド  
#concat：columns(列名)や index (行名)を参照して結合するメソッド。ignore_indexとはindexを無視するという意味。concatでデータフレームを結合した際にindexに付与された番号を無視すると言う事になる。
#drop dropで行を削除する  参考URL　https://www.sejuku.net/blog/73830

In [ ]:
#データのスクレイピングとデータ整形のコードは分ける。